In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import pandas as pd

from skimage import io
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
from skimage.color import label2rgb, rgb2gray
from skimage.filters import try_all_threshold, threshold_isodata, threshold_minimum, threshold_otsu, threshold_triangle


In [12]:
def BFCount (Filename = '14230.2_5(3)_BF.jpg', Test = False, QuickCheck = False):
    # This function count seeds in BF images
    # The default Filename is for testing 
    # 'Test' allows you to check all different threshold methods, normally used for individual images
    # 'QuickCheck' allows you to check the binary images current method ('minimum' in this case), 
    #  normally used for all images
    
    image = io.imread(Filename)
    gray_image = rgb2gray(image)
    
    # Set threshold using the smaller value of 'minimum' and 'triangle' method 
    thresh1 = threshold_minimum(gray_image)
    thresh2 = threshold_triangle(gray_image)
    thresh = min(thresh1,thresh2)
    binary = gray_image < thresh
    
    if Test == True:
        try_all_threshold(gray_image)
        return
    
    if QuickCheck == True:
        plt.figure()
        plt.imshow(binary)
        plt.title(Filename)
        return
    
    # Label regions
    bw = closing(binary, square(3)) 
    cleared = clear_border(bw) #ignore all the seeds at the edge of the image
    label_image = label(cleared)
    seed = 0
    cali_seed = 0
    cali_area = 0
    large_area = 0

    for region in regionprops(label_image):
        if 400 < region.area < 700 :
            cali_seed = cali_seed + 1
            cali_area = cali_area + region.area
        if 5 < region.area < 700:
            seed = seed + 1
        elif region.area > 700:
            large_area = large_area + region.area
    
    if cali_seed == 0:
        print('Calibrition of seed area failed: ' + Filename)
        plt.figure()
        plt.imshow(binary)
        plt.title(Filename)
        return 
    else:
        ave_area = cali_area / cali_seed
        large_seed = large_area / ave_area
        seed = seed + large_seed
        return int(seed)


In [13]:
def FluCount (Filename = '14251.5_3(2).jpg', Test = False, QuickCheck = False, Channel = 'G'):
    # This function count seeds in fluorescent images
    # The default Filename is for testing 
    # 'Test' allows you to check all different threshold methods, normally used for individual images
    # 'QuickCheck' allows you to check the binary images current method ('minimum' in this case), 
    # normally used for checking all images.
    # 'Channel' defines whether to use Green or Red channels for GFP and RFP seeds
    
    
    image = io.imread(Filename)
    
    
    #-----------------------------------Split channels----------------------------------------------
    
    if Channel == 'G':
        # Change to grayscale according to green or red channel
        green_image = image.copy()
        green_image[:,:,0] = 0
        green_image[:,:,2] = 0
        gray_image = rgb2gray(green_image)
        
    else:
        red_image = image.copy()
        red_image[:,:,1] = 0
        red_image[:,:,2] = 0
        gray_image = rgb2gray(red_image)
    
    # Set threshold to get binary images (for all fluorescent seeds)
    
    thresh = threshold_otsu(gray_image)
    binary = gray_image > thresh
    
    if QuickCheck == 1:
        plt.figure()
        plt.imshow(binary)
        plt.title(Filename + 'Fluorescent seeds')
        return
    
    # Label regions
    bw = closing(binary, square(3)) 
    cleared = clear_border(bw) #ignore all the seeds at the edge of the image
    label_image = label(cleared)
    seed = 0
    cali_seed = 0
    cali_area = 0
    large_area = 0

    for region in regionprops(label_image):
        if 400 < region.area < 700 :
            cali_seed = cali_seed + 1
            cali_area = cali_area + region.area
        if 5 < region.area < 700:
            seed = seed + 1
        elif region.area > 700:
            large_area = large_area + region.area

    ave_area = cali_area / cali_seed
    large_seed = large_area / ave_area
    seed = seed + large_seed

    #------------------------------------------------------------------------------------
    #Now for the bright seeds (overwrite the above images)
    
    # Set threshold to get binary images (for bright seeds)
    
    
    seed_Bright = 0
    try:
        thresh = threshold_minimum(gray_image)
    except:
        print('Problems with setting the threshold for counting bright seeds: ' + str(Filename)) 
        if Test == True:
            try_all_threshold(gray_image)
    else:
        binary = gray_image > thresh
        
        if QuickCheck == 2:
            plt.figure()
            plt.imshow(binary)
            plt.title(Filename + 'Bright seeds')
            return
        
        # Label regions
        bw = closing(binary, square(3)) 
        cleared = clear_border(bw) #ignore all the seeds at the edge of the image
        label_image = label(cleared)

        seed_Bright = 0
        cali_seed_Bright = 0
        cali_area_Bright = 0     
        large_area_Bright = 0

        for region in regionprops(label_image):
            if 400 < region.area < 700 :
                cali_seed_Bright = cali_seed_Bright + 1
                cali_area_Bright = cali_area_Bright + region.area
            if 5 < region.area < 700:
                seed_Bright = seed_Bright + 1
            elif region.area > 700:
                large_area_Bright = large_area_Bright + region.area

        if cali_seed_Bright <20:
            ave_area_Bright = ave_area
        else:
            ave_area_Bright = cali_area_Bright / cali_seed_Bright
        large_seed_Bright = large_area_Bright / ave_area_Bright
        seed_Bright = seed_Bright + large_seed_Bright
    
    return int(seed), int(seed_Bright)

    

## Counting Funtions

In [14]:
def Give_Me_BFCounts(File = '*_BF.jpg', QuickCheck = False, Test = False):
    # This function takes a group of files as input, calculate the seeds and combine the number per 
    # consrtucts. Output is a .csv file
    # I suggest you first run a quick check of the binary images you create from thresholding methods, to do this,
    # use 'QuickCheck = True'
    
    if QuickCheck == True: 
        for file in glob.glob(File):
            BFCount(file, QuickCheck = True)
        return

    OutBF = []
    for file in glob.glob(File):
        dic = {'File':file, 'Seed':BFCount(file,Test)}
        OutBF.append(dic)

    df = pd.DataFrame(OutBF, columns=OutBF[0].keys())

    FileList = df['File']

    ConsList = []
    for i in FileList:
        ConsList.append(i.split('(')[0])
    ConsList = list(set(ConsList))

    CleanDf = []
    for i in ConsList:
        sub = df[df['File'].str.match(str(i))]
        dic = {'Construct' : str(i), 'Total Seed': sub['Seed'].sum()}
        CleanDf.append(dic)

    Counts = pd.DataFrame(CleanDf, columns=CleanDf[0].keys())
    return Counts

   

In [15]:
def Give_Me_FluCounts(File = '*).jpg', QuickCheck = False, Test = False, Channel = 'G'):
    # This function takes a group of files as input, calculate the fluorescent seeds and combine the number per 
    # consrtucts. Output is a .csv file
    # I suggest you first run a quick check of the binary images you create from thresholding methods, to do this,
    # use 'QuickCheck = 1' to check all fluorescent seeds, use 'QuickCheck = 2' to check all bright seeds
    
    if QuickCheck != False: 
        for file in glob.glob(File):
            FluCount(file, QuickCheck = True, Channel = Channel)
        return
    
    OutFlu = []
    for file in glob.glob(File):
        dic = {'File':file, 'Seed':FluCount(file,Test = False, Channel = Channel)[0], 
               'Bright_Seed':FluCount(file, Test, Channel = Channel)[1]}
        OutFlu.append(dic)

    df = pd.DataFrame(OutFlu, columns=OutFlu[0].keys())

    FileList = df['File']

    ConsList = []
    for i in FileList:
        ConsList.append(i.split('(')[0])
    ConsList = list(set(ConsList))

    CleanDf = []
    for i in ConsList:
        sub = df[df['File'].str.match(str(i))]
        dic = {'Construct' : str(i), 'Total fluorescent Seed': sub['Seed'].sum(), 
               'Bright Seed': sub['Bright_Seed'].sum()}
        CleanDf.append(dic)

    Counts = pd.DataFrame(CleanDf, columns=CleanDf[0].keys())
    return Counts
   

# All-in-One function

In [24]:
def Give_Me_Counts(QuickCheck = False, Test = False, Channel = 'G'):
    #This function uses the above functions and calculates everything, but make sure the files are named correctly
    
    BF = Give_Me_BFCounts(QuickCheck = QuickCheck, Test = Test)
    Flu = Give_Me_FluCounts(QuickCheck = QuickCheck, Test = Test, Channel = Channel)
    res = Flu.join(BF.set_index('Construct'), on='Construct')
    res['Rate'] = res['Total fluorescent Seed'] / res['Total Seed']
    res.to_csv('Counts.csv')

In [25]:
Give_Me_Counts()

Problems with setting the threshold for counting bright seeds: 14251.5_3(1).jpg
Problems with setting the threshold for counting bright seeds: 14251.5_3(1).jpg
Problems with setting the threshold for counting bright seeds: 14251.5_4(1).jpg
Problems with setting the threshold for counting bright seeds: 14251.5_4(1).jpg
